In the realm of large language models (LLMs) like OpenAI's GPT, chains and their variants provide a structured way to manage and execute complex tasks by leveraging multiple models or steps sequentially or conditionally. Here's an overview of these concepts and how they might be used:

### LLMChain

An LLMChain typically refers to a chain of tasks or queries that are executed sequentially, where the output of one task serves as the input for the next. This allows for complex workflows to be broken down into manageable steps.

### Sequential Chains

Sequential Chains are a specific type of LLMChain where tasks are executed in a strict order. There are two main types of Sequential Chains:

#### SimpleSequentialChain

A SimpleSequentialChain is a straightforward sequence of tasks, where each step's output directly feeds into the next step. It's useful for linear processes where each step builds on the previous one.

**Example Use Case:**
1. **Step 1:** Generate a summary of a document.
2. **Step 2:** Translate the summary into another language.
3. **Step 3:** Format the translated summary for a specific presentation style.

In this chain, the output of each step becomes the input for the next step, and there is no branching or decision-making involved.

#### SequentialChain

A SequentialChain is more flexible than a SimpleSequentialChain. It allows for conditional logic and branching within the sequence of tasks. This means that different paths can be taken based on the output or conditions met at each step.

**Example Use Case:**
1. **Step 1:** Classify a user query (e.g., as a question, command, or feedback).
2. **Step 2:** Based on the classification, route the query to different processing steps.
   - If it's a question, generate an answer.
   - If it's a command, execute the command.
   - If it's feedback, log the feedback and respond with an acknowledgment.

This type of chain can handle more complex workflows that require decision-making based on intermediate results.

### Router Chain

A Router Chain is designed to handle scenarios where different types of tasks need to be routed to different processing chains based on specific criteria. This involves using a router mechanism to determine which chain to execute based on the input.

**Example Use Case:**
- A support bot receives various types of user queries.
- The router chain determines the type of query (e.g., technical issue, billing question, general inquiry) and routes it to the appropriate processing chain.
  - Technical issues are routed to a chain that gathers more details and suggests troubleshooting steps.
  - Billing questions are routed to a chain that verifies account information and provides billing details.
  - General inquiries are routed to a chain that searches a knowledge base and provides relevant information.

Router Chains are highly useful in applications where there are multiple distinct workflows that need to be executed based on the nature of the input.

### Summary

- **LLMChain:** General concept of chaining tasks together.
- **SimpleSequentialChain:** Linear sequence of tasks with no branching.
- **SequentialChain:** Flexible sequence with conditional logic and branching.
- **Router Chain:** Routes tasks to different chains based on input criteria.

These chaining mechanisms provide powerful tools for managing complex interactions and workflows in applications leveraging large language models.

In [1]:
import os
from openai import OpenAI

llm_model = "gpt-3.5-turbo"

# Load openai api key
OPENAI_API_KEY_PATH = '/home/loc/Documents/OPENAI_API_KEY.txt'
with open(OPENAI_API_KEY_PATH) as f:
    OPENAI_API_KEY = f.read().strip()
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [3]:
import pandas as pd
df = pd.read_csv('Data.csv')
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


# LLMChain

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [5]:
llm = ChatOpenAI(temperature=0.9, model = llm_model)

In [6]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [7]:
chain = LLMChain(llm=llm, prompt=prompt)

In [8]:
product = "Queen Size Sheet Set"
chain.run(product)

'"Regal Linens"'

# SimpleSequentialChain

In [9]:
from langchain.chains import SimpleSequentialChain

In [11]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [12]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)

# Chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [14]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

In [15]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Regal Linens
Regal Linens offers luxurious and high-quality bedding and linens for a comfortable and stylish home decor experience. Upgrade your bedroom today.

> Finished chain.


'Regal Linens offers luxurious and high-quality bedding and linens for a comfortable and stylish home decor experience. Upgrade your bedroom today.'

# SequentialChain

In [16]:
from langchain.chains import SequentialChain

In [17]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [18]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

In [19]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

In [20]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [21]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [22]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message

overall_chain = SequentialChain(
    chains = [chain_one, chain_two, chain_three, chain_four],
    input_variables = ['Review'],
    output_variables = ["English_Review", "summary","followup_message"],
    verbose=True
)

In [23]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam does not hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer finds that the taste and foam of the product are subpar compared to what they typically purchase in stores, questioning if it is an old batch or counterfeit.',
 'followup_message': "Cher client, nous sommes désolés d'apprendre que vous n'avez pas été satisfait de notre produit. Nous vous assurons que nos produits sont authentiques et fabriqués avec les meilleurs ingrédients. Nous allons enquêter sur la question de savoir s'il s'agit d'un vieux lot et nous vous tiendrons informé. Nous apprécions vos commentaires et nous ferons de notre mieux pour améliorer votre expérience avec nos produits à l'avenir. Merci de nous

# Router Chain

In [34]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [35]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [36]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [37]:
destination_chains = {}

for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [38]:
destinations_str

'physics: Good for answering questions about physics\nmath: Good for answering math questions\nHistory: Good for answering history questions\ncomputer science: Good for answering computer science questions'

In [39]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [40]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [41]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [42]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [43]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized object that absorbs all radiation incident upon it and emits energy at every wavelength. The spectrum of black body radiation is continuous and depends only on the temperature of the object, following a specific curve known as Planck's law. This concept is important in understanding thermal radiation and plays a key role in various areas of physics, such as quantum mechanics and thermodynamics."

In [44]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer is 4.'

In [45]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'